# Function Calling API

In [12]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
OPENAI_API_KEY = os.getenv("genai_openai_key")
rapid_api_key = os.getenv("rapid_api_key")
openai.api_key = OPENAI_API_KEY

In [14]:
!pip show openai

Name: openai
Version: 1.63.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License-Expression: Apache-2.0
Location: C:\Users\balaj\AppData\Roaming\Python\Python311\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai


# Rapid API Call

In [16]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

import requests

def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://weather-api167.p.rapidapi.com/api/weather/forecast"
    querystring = {"place":location}

    headers = {
    'x-rapidapi-key': rapid_api_key,
    'x-rapidapi-host': "weather-api167.p.rapidapi.com",
	"Accept": "application/json"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())

    return response.json()

response = get_current_weather("Salem, Tamilnadu")

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1750561200, 'summery': 'light rain, 285.3K, feels like 284.95K, winds 2.82 meter/sec from South, 91% humidity', 'main': {'temprature': 285.3, 'temprature_feels_like': 284.95, 'temprature_min': 285.3, 'temprature_max': 286.06, 'temprature_unit': 'K', 'pressure': 1018, 'sea_level_pressure': 1018, 'ground_level_pressure': 1009, 'pressure_unit': 'hPa', 'humidity': 91, 'humidity_unit': '%', 'temp_kf': -0.76}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': 'https://openweathermap.org/img/wn/10d@2x.png'}], 'clouds': {'cloudiness': 100, 'unit': '%'}, 'wind': {'speed': 2.82, 'degrees': 173, 'direction': 'South', 'gust_speed': 4.84, 'speed_unit': 'meter/sec'}, 'visibility_distance': 10000, 'visibility_unit': 'm', 'probability_of_precipitation': 100, 'probability_of_precipitation_unit': '%', 'rain': {'amount': 0.85, 'unit': 'mm in the last 3 hour'}, 'snow': {'amount': 0, 'unit': 'mm in the last 3 hour'}, 'sys': 

In [43]:
response['list'][0]['summery']

'light rain, 285.3K, feels like 284.95K, winds 2.82 meter/sec from South, 91% humidity'

## Function Call

In [44]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                }
            },
            "required":["location"]
        }
    }
]

## Chat Completion API

In [74]:
user_message1 = "Hi, There"
user_message2 = "Wether of Salem, Tamilnadu"

messages = []
messages.append({"role":"user","content":user_message1})
messages.append({"role":"user","content":user_message2})

In [ ]:
response_1 = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [52]:
print(response_1.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"Salem, Tamilnadu"}', name='get_current_weather'), tool_calls=None, annotations=[])


In [67]:
function_detail = response_1.choices[0].message.function_call
location = eval(function_detail.arguments)['location']
function_name = function_detail.name

print(f"function_name = {function_name} \nlocation = {location}")

function_name = get_current_weather 
location = Salem, Tamilnadu


In [75]:
# Send these info to the function call  and function response to GPT

messages.append(response_1.choices[0].message)
messages.append(
    {
        "role":"function",
        "name": function_name,
        "content": location
    }
)
messages

[{'role': 'user', 'content': 'Hi, There'},
 {'role': 'user', 'content': 'Wether of Salem, Tamilnadu'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"Salem, Tamilnadu"}', name='get_current_weather'), tool_calls=None, annotations=[]),
 {'role': 'function',
  'name': 'get_current_weather',
  'content': 'Salem, Tamilnadu'}]

In [76]:
# extend the conversation with function response
second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [78]:
second_response.choices[0].message.content

'The current weather in Salem, Tamil Nadu is foggy with a temperature of 27°C.'